In [ ]:

import numpy as np
import cv2
def stereoMatchSGBM(left_image, right_image, down_scale=False):
    # SGBM匹配参数设置
    if left_image.ndim == 2:
        img_channels = 1
    else:
        img_channels = 3
    blockSize = 5
    paraml = {'minDisparity': 0,
             'numDisparities': 320,#SGBM认为图像左侧前numDisparities列是不能估计视差的，所以会看到视差图左侧有一块是黑色
             'blockSize': blockSize,
             'P1': 64 * img_channels * blockSize ** 2,
             'P2': 256 * img_channels * blockSize ** 2,# 加大P1、P2，进行弱纹理优化
             'disp12MaxDiff': 1,
             'preFilterCap': 63,
             'uniquenessRatio': 15,
             'speckleWindowSize': 100,
             'speckleRange': 1,
             'mode': cv2.STEREO_SGBM_MODE_SGBM_3WAY
             }
 
    # 构建SGBM对象
    left_matcher = cv2.StereoSGBM_create(**paraml)
    paramr = paraml
    paramr['minDisparity'] = -paraml['numDisparities']
    right_matcher = cv2.StereoSGBM_create(**paramr)
 
    # 计算视差图
    size = (left_image.shape[1], left_image.shape[0])
    if down_scale == False:
        disparity_left = left_matcher.compute(left_image, right_image)
        disparity_right = right_matcher.compute(right_image, left_image)
 
    else:# 降采样并插值处理图像，提高计算效率，并使图像更加平滑，减少图像噪声
        left_image_down = cv2.pyrDown(left_image)
        right_image_down = cv2.pyrDown(right_image)
        factor = left_image.shape[1] / left_image_down.shape[1]
 
        disparity_left_half = left_matcher.compute(left_image_down, right_image_down)
        disparity_right_half = right_matcher.compute(right_image_down, left_image_down)
        disparity_left = cv2.resize(disparity_left_half, size, interpolation=cv2.INTER_AREA)
        disparity_right = cv2.resize(disparity_right_half, size, interpolation=cv2.INTER_AREA)
        disparity_left = factor * disparity_left
        disparity_right = factor * disparity_right
 
    # 真实视差（因为SGBM算法得到的视差是×16的）
    trueDisp_left = disparity_left.astype(np.float32) / 16.
    trueDisp_right = disparity_right.astype(np.float32) / 16.
 
    return trueDisp_left, trueDisp_right,left_matcher,right_matcher

if __name__=='__main__':
    #读取图像
    melonL = cv2.imread('./img/left1.jpg',0)
    melonR = cv2.imread('./img/right1.jpg',0)

    disp0, disp1,matcher0,matcher1=stereoMatchSGBM(melonL,melonR,True)

    #创建滤波器
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher0)
    wls_filter.setLambda(80000)
    wls_filter.setSigmaColor(1.3)

    #滤波器平滑视差图
    depth1 = wls_filter.filter(disp0, melonL, None, disp1).astype(np.float32)/16.
    depth2 = wls_filter.filter(disp1, melonL, None, disp0).astype(np.float32)/16.

    depth1 = np.uint8(cv2.normalize(depth1, depth1, 255, 0, cv2.NORM_MINMAX))
    depth2 = np.uint8(cv2.normalize(depth2, depth2, 255, 0, cv2.NORM_MINMAX))
    cv2.imwrite('./img/Disparity1_1.jpg', depth1)
    cv2.imwrite('./img/Disparity1_2.jpg', depth2)